<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/22520975_Lab2_Bai2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import os
import time
import copy
from tqdm.auto import tqdm

In [2]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [3]:
# --- Cấu hình chung ---
DATA_DIR = '/content/drive/MyDrive/Data/VinaFood21/VinaFood21'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

if not os.path.exists(DATA_DIR):
    print(f"LỖI: Không tìm thấy thư mục '{DATA_DIR}'.")
    print("Vui lòng kiểm tra lại đường dẫn Google Drive của bạn.")

NUM_CLASSES = 21 # VinaFood21

Sử dụng thiết bị: cuda:0


In [4]:
def get_dataloaders(img_size, batch_size=32):
    """Tải dữ liệu với kích thước ảnh cụ thể."""
    print(f"\nĐang tải dữ liệu với kích thước {img_size}x{img_size}...")
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    try:
        image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                          for x in ['train', 'test']}
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x=='train'), num_workers=2)
                       for x in ['train', 'test']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
        class_names = image_datasets['train'].classes
        print(f"Tải xong. {dataset_sizes['train']} ảnh train, {dataset_sizes['test']} ảnh test.")
        return dataloaders, dataset_sizes, class_names
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy thư mục 'train' hoặc 'test' trong {DATA_DIR}")
        return None, None, None

In [5]:
def quick_evaluate(model, dataloader):
    """Hàm đánh giá nhanh (chỉ tính Acc) để xem sau mỗi epoch."""
    model.eval()
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
    acc = running_corrects.double() / total
    print(f'Test Acc (Validation): {acc:.4f}')
    model.train() # Chuyển lại chế độ train

In [6]:
def train_model_torchvision(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện cho LeNet và ResNet (output chuẩn)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])


    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [7]:
def train_model_googlenet(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện đặc biệt cho GoogLeNet (có 3 output)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                main_out, aux1_out, aux2_out = model(inputs)
                loss_main = criterion(main_out, labels)
                loss_aux1 = criterion(aux1_out, labels)
                loss_aux2 = criterion(aux2_out, labels)
                loss = loss_main + 0.3 * (loss_aux1 + loss_aux2)
                _, preds = torch.max(main_out, 1)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])

    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [8]:
def evaluate_model_metrics(model, dataloader):
    """Đánh giá mô hình trên tập test và tính Precision, Recall, F1-macro."""
    print("\nĐang đánh giá chi tiết trên tập Test...")
    model = model.to(device)
    model.eval() # Chuyển sang chế độ eval

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels,
        all_preds,
        average='macro',
        zero_division=0
    )
    accuracy = accuracy_score(all_labels, all_preds)

    print("\n--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---")
    print(f'Accuracy:  {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall:    {recall:.4f}')
    print(f'F1-Score:  {f1:.4f}')
    print("-" * 39)

In [9]:
# --- Tải Dữ Liệu ---
dataloaders_28, sizes_28, classes_28 = get_dataloaders(img_size=28, batch_size=64)
dataloaders_224, sizes_224, classes_224 = get_dataloaders(img_size=224, batch_size=32)


Đang tải dữ liệu với kích thước 28x28...
Tải xong. 10089 ảnh train, 6693 ảnh test.

Đang tải dữ liệu với kích thước 224x224...
Tải xong. 10089 ảnh train, 6693 ảnh test.


In [10]:
# -------------------------------------------------------------------
# BÀI 2: GoogLeNet (224x224)
# -------------------------------------------------------------------
print("\n" + "="*50)
print("BẮT ĐẦU BÀI 2: GoogLeNet")
print("="*50)

# 1. Định nghĩa Khối Inception
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, kernel_size=1),
            nn.BatchNorm2d(ch1x1), nn.ReLU(inplace=True)
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1),
            nn.BatchNorm2d(ch3x3red), nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(ch3x3), nn.ReLU(inplace=True)
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1),
            nn.BatchNorm2d(ch5x5red), nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(ch5x5), nn.ReLU(inplace=True)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1),
            nn.BatchNorm2d(pool_proj), nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

# 2. Bộ phân loại phụ
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(4 * 4 * 128, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x); x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x); x = self.relu(x); x = self.dropout(x)
        x = self.fc2(x)
        return x

# 3. Kiến trúc GoogLeNet
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=21, aux_logits=True):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), nn.BatchNorm2d(64), nn.ReLU(inplace=True))
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.BatchNorm2d(192), nn.ReLU(inplace=True))
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x); x = self.maxpool1(x); x = self.conv2(x);
        x = self.conv3(x); x = self.maxpool2(x)
        x = self.inception3a(x); x = self.inception3b(x); x = self.maxpool3(x)
        x = self.inception4a(x)

        aux1_out = None
        if self.training and self.aux_logits:
            aux1_out = self.aux1(x)

        x = self.inception4b(x); x = self.inception4c(x); x = self.inception4d(x)

        aux2_out = None
        if self.training and self.aux_logits:
            aux2_out = self.aux2(x)

        x = self.inception4e(x); x = self.maxpool4(x)
        x = self.inception5a(x); x = self.inception5b(x)
        x = self.avgpool(x); x = x.view(x.size(0), -1); x = self.dropout(x)
        main_out = self.fc(x)

        if self.training and self.aux_logits:
            return main_out, aux1_out, aux2_out
        else:
            return main_out

if dataloaders_224:
    model_googlenet = GoogLeNet(num_classes=NUM_CLASSES, aux_logits=True).to(device)
    criterion_g = nn.CrossEntropyLoss()
    optimizer_g = optim.Adam(model_googlenet.parameters(), lr=0.001)

    NUM_EPOCHS_B2 = 50
    print(f"Huấn luyện GoogLeNet trong {NUM_EPOCHS_B2} epochs...")

    model_googlenet_trained = train_model_googlenet(
        model_googlenet, dataloaders_224, sizes_224,
        criterion_g, optimizer_g, num_epochs=NUM_EPOCHS_B2
    )

    print("\n[Bài 2: GoogLeNet] Đánh giá cuối cùng:")
    evaluate_model_metrics(model_googlenet_trained, dataloaders_224['test'])
else:
    print("Bỏ qua Bài 2 do không tải được dữ liệu.")


BẮT ĐẦU BÀI 2: GoogLeNet
Huấn luyện GoogLeNet trong 50 epochs...
Epoch 1/50
----------


Training Epoch 1:   0%|          | 0/316 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 4.2950 Acc: 0.1794
Test Acc (Validation): 0.1809
Epoch 2/50
----------


Training Epoch 2:   0%|          | 0/316 [00:00<?, ?it/s]

...

Training Epoch 50:   0%|          | 0/316 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d3380dec220>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d3380dec220>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packa

Train Loss: 0.2693 Acc: 0.9625
Test Acc (Validation): 0.6314

Huấn luyện hoàn tất trong 228m 3s

[Bài 2: GoogLeNet] Đánh giá cuối cùng:

Đang đánh giá chi tiết trên tập Test...


Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]


--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---
Accuracy:  0.6314
Precision: 0.6532
Recall:    0.6169
F1-Score:  0.6124
---------------------------------------
